## LINEAR_REG Prédire le prix des courses de Taxi NYC

In [1]:
%%bigquery
SELECT
  pickup_datetime,
  dropoff_datetime,
  passenger_count,
  trip_distance,
  fare_amount,
  tip_amount,
  total_amount
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount
0,2015-12-10 23:40:11+00:00,2015-12-10 23:40:11+00:00,1,0E-9,0E-9,0E-9,0E-9
1,2015-12-15 12:35:28+00:00,2015-12-15 12:35:28+00:00,1,0E-9,0E-9,0E-9,0E-9
2,2015-12-16 22:37:02+00:00,2015-12-16 22:37:02+00:00,3,0E-9,0E-9,0E-9,0E-9
3,2015-12-19 03:00:30+00:00,2015-12-19 03:00:30+00:00,1,0E-9,0E-9,0E-9,0E-9
4,2015-12-08 09:01:14+00:00,2015-12-08 09:45:45+00:00,3,28.110000000,0E-9,0E-9,0E-9
5,2015-12-15 23:37:04+00:00,2015-12-15 23:37:04+00:00,1,0E-9,0E-9,0E-9,0E-9
6,2015-12-04 23:04:35+00:00,2015-12-04 23:04:35+00:00,1,0E-9,0E-9,0E-9,0E-9
7,2015-12-20 19:16:53+00:00,2015-12-20 19:16:53+00:00,1,0E-9,0E-9,0E-9,0E-9
8,2015-12-02 19:43:33+00:00,2015-12-02 19:43:33+00:00,1,0E-9,0E-9,0E-9,0E-9
9,2015-12-14 20:55:39+00:00,2015-12-14 22:55:39+00:00,0,0E-9,0E-9,0E-9,0E-9


In [2]:
%%bigquery
SELECT
  COUNT(*) AS nombre_courses,
  ROUND(AVG(fare_amount), 2) AS prix_moyen,
  ROUND(MIN(fare_amount), 2) AS prix_min,
  ROUND(MAX(fare_amount), 2) AS prix_max,
  ROUND(AVG(trip_distance), 2) AS distance_moyenne,
  ROUND(AVG(passenger_count), 1) AS passagers_moyen
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
WHERE fare_amount > 0 AND trip_distance > 0;

Query is running:   0%|          |

Downloading:   0%|          |

,nombre_courses,prix_moyen,prix_min,prix_max,distance_moyenne,passagers_moyen
0,145098861,12.880000000,0.010000000,503325.530000000,12.480000000,1.7


In [3]:
%%bigquery
SELECT
  'Prix < 0 ou > 200' AS probleme,
  COUNT(*) AS nombre
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
WHERE fare_amount < 0 OR fare_amount > 200

UNION ALL

SELECT
  'Distance = 0',
  COUNT(*)
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
WHERE trip_distance = 0 OR trip_distance IS NULL

UNION ALL

SELECT
  'Passagers = 0 ou > 6',
  COUNT(*)
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
WHERE passenger_count = 0 OR passenger_count > 6;

Query is running:   0%|          |

Downloading:   0%|          |

,probleme,nombre
0,Distance = 0,876724
1,Prix < 0 ou > 200,60611
2,Passagers = 0 ou > 6,41253


In [4]:
%%bigquery
CREATE OR REPLACE TABLE `ml_models.taxi_data_clean` AS
SELECT
  trip_distance,
  passenger_count,
  fare_amount
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
WHERE
  fare_amount BETWEEN 1 AND 200
  AND trip_distance BETWEEN 0.1 AND 50
  AND passenger_count BETWEEN 1 AND 6
LIMIT 500000;

Query is running:   0%|          |

""


In [5]:
%%bigquery
SELECT
  COUNT(*) AS lignes,
  ROUND(AVG(fare_amount), 2) AS prix_moyen,
  ROUND(AVG(trip_distance), 2) AS distance_moyenne
FROM `ml_models.taxi_data_clean`;

Query is running:   0%|          |

Downloading:   0%|          |

,lignes,prix_moyen,distance_moyenne
0,500000,12.560000000,3.030000000


## Créer le modèle

In [6]:
%%bigquery
CREATE OR REPLACE MODEL `ml_models.taxi_price_v1_ihab`
OPTIONS(
  model_type='LINEAR_REG',
  input_label_cols=['fare_amount']
) AS
SELECT
  trip_distance,
  passenger_count,
  fare_amount
  FROM `ml_models.taxi_data_clean`;

Query is running:   0%|          |

""


In [9]:
%%bigquery
SELECT
  mean_absolute_error,
  mean_squared_error,
  r2_score
  FROM ML.EVALUATE(MODEL `ml_models.taxi_price_v1_ihab`);

Query is running:   0%|          |

Downloading:   0%|          |

,mean_absolute_error,mean_squared_error,r2_score
0,1.57822,9.573959,0.916733


In [11]:
%%bigquery
SELECT
  trip_distance,
  passenger_count,
  predicted_fare_amount as prix_predit
  FROM ML.PREDICT (
    MODEL `ml_models.taxi_price_v1_ihab`,
    (
      SELECT
      8 AS trip_distance,
      3 AS passenger_count
    )
  )

Query is running:   0%|          |

Downloading:   0%|          |

,trip_distance,passenger_count,prix_predit
0,8,3,25.985558


In [14]:
%%bigquery
SELECT
  processed_input AS variable,
  ROUND(weight, 4) AS coefficient,
FROM ML.WEIGHTS(MODEL `ml_models.taxi_price_v1_ihab`)
ORDER BY ABS(weight) DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,variable,coefficient
0,__INTERCEPT__,4.4004
1,trip_distance,2.7028
2,passenger_count,-0.0123
